In [1]:
import os
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
from PIL import Image
from matplotlib.ticker import FormatStrFormatter
import torchvision.models as models

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

from collections import OrderedDict
import torch.optim.lr_scheduler as lr_scheduler

import io
import pathlib
from torch.autograd import Variable
import seaborn as sns



In [2]:
import os

source = os.listdir("H:/deep learning/asl-alphabet/asl_alphabet_train/")
destination = os.listdir("H:/deep learning/asl-alphabet/asl_alphabet_valid/")

In [3]:
import shutil

In [4]:
fn_source = "H:/deep learning/asl-alphabet/asl_alphabet_train/"
fn_destination = "H:/deep learning/asl-alphabet/asl_alphabet_valid/"

alpha = os.listdir(fn_source+source[0])[0]
(fn_destination+destination[0])+'/'

'H:/deep learning/asl-alphabet/asl_alphabet_valid/A/'

In [ ]:
for i in range(len(source)):
    for k in range(200):
        if (os.listdir(fn_source+source[i])[k]).endswith(".jpg"):
            shutil.move(os.listdir(fn_source+source[i])[k] , (fn_destination+destination[i]+'/'))

In [ ]:
for i in source[:200]:
    for k in range(200):
        if os.listdir(fn_source+source[i]).endswith(".jpg"):
            shutil.move(os.listdir(fn_source+source[i]), (fn_destination+destination[i]))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
data_dir = "H:/deep learning/asl-alphabet/"
train_dir = data_dir + "asl_alphabet_train"
valid_dir = data_dir + "asl_alphabet_valid"

In [7]:
transforms = transforms.Compose([
                                      transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.Grayscale(num_output_channels=3),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])

train_image_datasets = datasets.ImageFolder(train_dir,transform=transforms)
valid_image_datasets = datasets.ImageFolder(valid_dir,transform=transforms)

batch_size = 32
num_workers = 4

train_loader = torch.utils.data.DataLoader(train_image_datasets, batch_size=batch_size,shuffle=True,
                                           num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(valid_image_datasets,  batch_size=batch_size,shuffle=True,
                                           num_workers=num_workers)

In [8]:
model = models.vgg19_bn(pretrained=True)

for param in model.parameters():
    param.requires_grad =False
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [10]:
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                                ('drop1', nn.Dropout(p=0.2)),
                                ('fc1', nn.Linear(25088,4096)),
                                ('bn1', nn.BatchNorm1d(4096)),
                                ('relu1', nn.ReLU()),
                                ('drop2', nn.Dropout(p=0.3)),
                                ('fc2', nn.Linear(4096,512)),
                                ('bn2', nn.BatchNorm1d(512)),
                                ('relu2', nn.ReLU()),
                                ('fc3', nn.Linear(512,29)),
                                ('output', nn.LogSoftmax(dim=1))
                                ]))

model.classifier = classifier

In [11]:
import torch.optim.lr_scheduler as lr_scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1)

In [12]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3)

In [ ]:
n_epochs = 15

valid_loss_min = np.Inf 

for epoch in range(1, n_epochs+1):
    
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0
    scheduler.step(valid_loss)

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = criterion(output, target)
        
        loss.backward()
        
        optimizer.step()
    
        train_loss += loss.item()*data.size(0)
        
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        

        data, target = data.to(device), target.to(device)
        
        output = model(data)
        
        loss = criterion(output, target)
         
        valid_loss += loss.item()*data.size(0)
        
        ps = torch.exp(output)
        _, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()*data.size(0)
    
    
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    acc = accuracy/len(valid_loader.dataset) 
        
     
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {:.3f}'.format(
        epoch, train_loss, valid_loss, acc))
    
    
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_mi
            
            
            
            
            
            
            
                    
            
            
            +  \
            
            
            
               
            
            
            
            
                
            
            
            
            n,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented_asl_densenet161.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.250515 	Validation Loss: 0.167897 	Accuracy: 0.946
Validation loss decreased (inf --> 0.167897).  Saving model ...
Epoch: 2 	Training Loss: 0.036812 	Validation Loss: 0.158810 	Accuracy: 0.950
Validation loss decreased (0.167897 --> 0.158810).  Saving model ...
Epoch: 3 	Training Loss: 0.018225 	Validation Loss: 0.170869 	Accuracy: 0.953
Epoch: 4 	Training Loss: 0.014520 	Validation Loss: 0.163906 	Accuracy: 0.957
Epoch: 5 	Training Loss: 0.009780 	Validation Loss: 0.199229 	Accuracy: 0.949
